# 환경 설정

In [38]:
# 가게 전체 평점 점수 크롤링
element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.restaurant_title_wrap > span > strong > span'
total_raw = driver.find_element(By.CSS_SELECTOR,element)
total = total_raw.text
print(total, type(total))

4.7 <class 'str'>


In [39]:
# 조회수 크롤링
try:
    element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.status.branch_none > span.cnt.hit'
    view_raw = driver.find_element(By.CSS_SELECTOR,element)
    view = view_raw.text
except:
    element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.status > span.cnt.hit'
    view_raw = driver.find_element(By.CSS_SELECTOR,element)
    view = view_raw.text
    
print(view, type(view))

235,714 <class 'str'>


In [54]:
# 리뷰 개수 크롤링
try:
    element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.status.branch_none > span.cnt.review'
    review_raw = driver.find_element(By.CSS_SELECTOR,element)
    num_review = review_raw.text

except:
    element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.status > span.cnt.review'
    review_raw = driver.find_element(By.CSS_SELECTOR,element)
    num_review = review_raw.text

print(num_review, type(num_review))

103 <class 'str'>


In [40]:
# 별표 개수 크롤링
try:
    element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.status.branch_none > span.cnt.favorite'
    star_raw =  driver.find_element(By.CSS_SELECTOR, element)
    num_star = star_raw.text

except:
    element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.status > span.cnt.favorite'
    star_raw =  driver.find_element(By.CSS_SELECTOR, element)
    num_star = star_raw.text

print(num_star, type(num_star))

5,030 <class 'str'>


In [50]:
# 음식 종류 및 메뉴 크롤링
element1 = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > table > tbody > tr:nth-child(3) > td > span'
type_raw = driver.find_element(By.CSS_SELECTOR,element1)
type_raw = type_raw.text
food_type = type_raw

'''
menu_list = list()

element2 = 'span.Restaurant_Menu'
menu_raw = driver.find_elements(By.CSS_SELECTOR,element2)
for i in menu_raw:
    menu_list.append(i.text)
food_type = [type_raw, menu_list]
'''

print(food_type)

['해산물 요리', ['고등어회 (대) + 탕', '고등어회 (소) + 탕', '고등어구이']]


In [47]:
list1 = driver.find_elements(By.CSS_SELECTOR, 'span.Restaurant_Menu')
for i in list1:
    print(i.text)

고등어회 (대) + 탕
고등어회 (소) + 탕
고등어구이


In [57]:
# 데이터 프레임 만들기
a = {'Title' : title,
     'Point' : total,
     'View' : view,
     'Review' : num_review,
     'Star' : num_star,
     'Type' : food_type}

df = pd.DataFrame.from_dict(a, orient='index')
df = df.transpose()
df.head()

,Title,Point,View,Review,Star,Type
0,미영이네식당,4.7,"235,415",103,"5,026","[해산물 요리, 고등어회 (대) + 탕\n77,000원\n고등어회 (소) + 탕\n..."


# for문으로 크롤링 : 1페이지만

In [1]:
# 필요한 라이브러리를 임포트하기

import sys # 시스템
import os  # 시스템

# 데이터 다루기
import pandas as pd
import numpy as np

# selenium 크롤링
from selenium import webdriver  
from selenium.webdriver import ActionChains as AC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

# 크롬 드라이버
#import chromedriver_autoinstaller

# beautifulsoup 크롤링
import requests
from bs4 import BeautifulSoup

# lxml 크롤링
import lxml.html

# 시간 조절
import time

# 시간 측정
from tqdm import notebook

# 정규표현식
import re

# 경고 무시
import warnings
warnings.filterwarnings('ignore')

In [2]:
# 지역 키워드를 입력하기
keyword = '횟집'

# page 정하기
page = 1

In [3]:
# 크롬창 띄우기
driver = webdriver.Chrome('./chromedriver')

# 키워드가 입력된 망고플레이트 사이트에 들어가기
driver.get(" https://www.mangoplate.com/search/{0}?keyword={0}&page={1}".format(keyword, page)) 
time.sleep(2)

In [4]:
# 광고창 iframe 접근하기
driver.switch_to.frame("google_ads_iframe_/395211568/init/desktop_all_0")

# '다시 보지 않기' 버튼 클릭하기
element = '#ad > div > button.ad_btn.ad_block_btn'
driver.find_element(By.CSS_SELECTOR, element).click()

In [5]:
driver.switch_to.default_content()

In [6]:
# 음식점 1개에서 정보 크롤링 함수
def FoodCrawling(row, col):
    # 전체 담을 그릇
    total_dict = {}
    k = 0
    
    for i in range(row):
        driver.execute_script("window.scrollTo(0, 200)")
        for j in range(col):
            # 동영상 1개 담을 그릇
            sub_dict = {}
            
            # 음식점 클릭
            element = f'body > main > article > div.column-wrapper > div > div > section > div.search-list-restaurants-inner-wrap > ul > li:nth-child({i+1}) > div:nth-child({j+1}) > figure > a > div > img'
            driver.find_element(By.CSS_SELECTOR,element).click()
            time.sleep(0.5)

            # 가게 이름 크롤링
            element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.restaurant_title_wrap > span > h1'
            title_raw = driver.find_element(By.CSS_SELECTOR, element)
            title = title_raw.text

            # 가게 전체 평점 점수 크롤링
            element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.restaurant_title_wrap > span > strong > span'
            total_raw = driver.find_element(By.CSS_SELECTOR,element)
            total = total_raw.text
            
            # 조회수 크롤링
            try:
                element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.status.branch_none > span.cnt.hit'
                view_raw = driver.find_element(By.CSS_SELECTOR,element)
                view = view_raw.text
            except:
                element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.status > span.cnt.hit'
                view_raw = driver.find_element(By.CSS_SELECTOR,element)
                view = view_raw.text

            # 리뷰 개수 크롤링
            try:
                element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.status.branch_none > span.cnt.review'
                review_raw = driver.find_element(By.CSS_SELECTOR,element)
                num_review = review_raw.text

            except:
                element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.status > span.cnt.review'
                review_raw = driver.find_element(By.CSS_SELECTOR,element)
                num_review = review_raw.text

            # 별표 개수 크롤링
            try:
                element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.status.branch_none > span.cnt.favorite'
                star_raw = driver.find_element(By.CSS_SELECTOR,element)
                num_star = star_raw.text

            except:
                element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.status > span.cnt.favorite'
                star_raw = driver.find_element(By.CSS_SELECTOR,element)
                num_star = star_raw.text

            try:
                # 음식 종류 및 메뉴 크롤링
                element1 = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > table > tbody > tr:nth-child(3) > td > span'
                food_type = driver.find_element(By.CSS_SELECTOR,element1)
                food_type = food_type.text
                #food_type = type_raw
            except:
                pass
            
            try:
                element2 = 'span.Restaurant_Menu'
                menu_elements = driver.find_elements(By.CSS_SELECTOR,element2)
                food_menu = [menu_element.text for menu_element in menu_elements]
            except:
                pass
            
            # sub_dict에 담기
            sub_dict['title'] = title
            sub_dict['Point'] = total
            sub_dict['View'] = view
            sub_dict['Review'] = num_review
            sub_dict['Star'] = num_star
            sub_dict['Type'] = food_type
            sub_dict['Menu'] = food_menu
            
            
             # total_dict에 담기
            total_dict[k] = sub_dict
            k += 1
            time.sleep(1)
            
            # 뒤로 가기
            driver.back()
            time.sleep(0.5)
            
    df = pd.DataFrame.from_dict(total_dict, orient='index')
    return df

In [7]:
'''
menu_list = list()

element2 = 'span.Restaurant_Menu'
menu_raw = driver.find_elements(By.CSS_SELECTOR,element2)
for i in menu_raw:
    menu_list.append(i.text)
food_type = [type_raw, menu_list]
'''

"\nmenu_list = list()\n\nelement2 = 'span.Restaurant_Menu'\nmenu_raw = driver.find_elements(By.CSS_SELECTOR,element2)\nfor i in menu_raw:\n    menu_list.append(i.text)\nfood_type = [type_raw, menu_list]\n"

In [8]:
df = FoodCrawling(row = 5, col = 2)
df

,title,Point,View,Review,Star,Type,Menu
0,미영이네식당,4.7,"236,463",103,"5,034",해산물 요리,"[고등어회 (대) + 탕, 고등어회 (소) + 탕, 고등어구이]"
1,남양수산,4.6,"396,438",112,"6,526",해산물 요리,"[참돔 (소), 참돔 (중), 참돔 (대), 전어, 활고등어]"
2,곰막,4.4,"191,937",91,"4,796",해산물 요리,"[회국수, 성게국수, 고등어구이, 전복회, 회덮밥]"
3,바다회사랑,4.4,"87,791",78,"2,238",해산물 요리,"[꽃멍게, 전어회(2인), 광어+연어 (소), 모듬회 (1인분), 모듬회 (소)]"
4,어시장전주식당,4.4,"36,179",18,"1,042",해산물 요리,[모듬회 (1인)]
5,마라도 회식당,4.4,"50,185",28,584,해산물 요리,"[최강달인물회, 모듬회 (중), 자연산모듬회 (중), 참가자미물회, 전복물회]"
6,표선어촌식당,4.4,"52,721",20,973,해산물 요리,"[갈치조림 (소), 우럭조림 (소), 옥돔구이, 쥐치매운탕, 해물뚝배기]"
7,전라상회,4.4,"50,223",26,"1,033",해산물 요리,"[전라 오마카세 (3인), 전라모듬회 특수부위 스페셜 (중), 감성돔 스페셜 (중)..."
8,바다회사랑,4.3,"266,727",166,"5,224",해산물 요리,"[광어 (소), 광어 (중), 광어 (대), 우럭 (중), 우럭 (대)]"
9,형제상회,4.3,"427,770",117,"6,390",해산물 요리,"[모듬회 (1인), 모듬회 (2인), 모듬회 (3인), 모듬회 (4인)]"


In [77]:
df.Type

0    [해산물 요리, 고등어회 (대) + 탕\n77,000원\n고등어회 (소) + 탕\n...
1                                               해산물 요리
2    [해산물 요리, 회국수\n11,000원\n성게국수\n12,000원\n고등어구이\n1...
3                                               해산물 요리
4                          [해산물 요리, 모듬회 (1인)\n25,000원]
5                                               해산물 요리
6                                               해산물 요리
7    [해산물 요리, 전라 오마카세 (3인)\n105,000원\n전라모듬회 특수부위 스페...
8                                               해산물 요리
9                                               해산물 요리
Name: Type, dtype: object

In [ ]:
# 필요한 라이브러리를 임포트하기

import sys # 시스템
import os  # 시스템

# 데이터 다루기
import pandas as pd
import numpy as np

# selenium 크롤링
from selenium import webdriver  
from selenium.webdriver import ActionChains as AC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

# 크롬 드라이버
#import chromedriver_autoinstaller

# beautifulsoup 크롤링
import requests
from bs4 import BeautifulSoup

# lxml 크롤링
import lxml.html

# 시간 조절
import time

# 시간 측정
from tqdm import notebook

# 정규표현식
import re

# 경고 무시
import warnings
warnings.filterwarnings('ignore')

In [95]:
def page_up(nums, driver):

IndentationError: expected an indented block (2429569189.py, line 1)

In [91]:
def review_crawling(driver):
    driver.execute_script("window.scrollTo(0, 800)")
    element = 'div.RestaurantReviewList__MoreReviewButton'
    driver.find_element(By.CSS_SELECTOR, element).click()
    time.sleep(0.5)
    
    

In [92]:
# 음식점 1개에서 정보 크롤링 함수
def FoodCrawling(row, col):
    # 전체 담을 그릇
    total_dict = {}
    k = 0
    
    for i in range(row):
        driver.execute_script("window.scrollTo(0, 200)")
        for j in range(col):
            # 동영상 1개 담을 그릇
            sub_dict = {}
            
            # 음식점 클릭
            element = f'body > main > article > div.column-wrapper > div > div > section > div.search-list-restaurants-inner-wrap > ul > li:nth-child({i+1}) > div:nth-child({j+1}) > figure > a > div > img'
            driver.find_element(By.CSS_SELECTOR,element).click()
            time.sleep(0.5)

            # 가게 이름 크롤링
            element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.restaurant_title_wrap > span > h1'
            title_raw = driver.find_element(By.CSS_SELECTOR, element)
            title = title_raw.text

            # 가게 전체 평점 점수 크롤링
            element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.restaurant_title_wrap > span > strong > span'
            total_raw = driver.find_element(By.CSS_SELECTOR,element)
            total = total_raw.text
            
            # 조회수 크롤링
            try:
                element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.status.branch_none > span.cnt.hit'
                view_raw = driver.find_element(By.CSS_SELECTOR,element)
                view = view_raw.text
            except:
                element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.status > span.cnt.hit'
                view_raw = driver.find_element(By.CSS_SELECTOR,element)
                view = view_raw.text

            # 리뷰 개수 크롤링
            try:
                element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.status.branch_none > span.cnt.review'
                review_raw = driver.find_element(By.CSS_SELECTOR,element)
                num_review = review_raw.text

            except:
                element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.status > span.cnt.review'
                review_raw = driver.find_element(By.CSS_SELECTOR,element)
                num_review = review_raw.text

            # 별표 개수 크롤링
            try:
                element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.status.branch_none > span.cnt.favorite'
                star_raw = driver.find_element(By.CSS_SELECTOR,element)
                num_star = star_raw.text

            except:
                element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.status > span.cnt.favorite'
                star_raw = driver.find_element(By.CSS_SELECTOR,element)
                num_star = star_raw.text

            # 음식 종류 
            try:
                element1 = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > table > tbody > tr:nth-child(3) > td > span'
                type_raw = driver.find_element(By.CSS_SELECTOR,element1)
                type_raw = type_raw.text
                food_type = type_raw
            except:
                element1 = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > table > tbody > tr:nth-child(2) > td > span'
                type_raw = driver.find_element(By.CSS_SELECTOR,element1)
                type_raw = type_raw.text
                food_type = type_raw
                
            # 메뉴 크롤링 
            try:
                element2 = 'span.Restaurant_Menu'
                menu_elements = driver.find_elements(By.CSS_SELECTOR,element2)
                food_menu = [menu_element.text for menu_element in menu_elements]
            except:
                pass
            
            # 리뷰 크롤링
            try :
                review_list = review_crawling(driver)
            except:
                pass
            # sub_dict에 담기
            sub_dict['title'] = title
            sub_dict['Point'] = total
            sub_dict['View'] = view
            sub_dict['Review'] = num_review
            sub_dict['Star'] = num_star
            sub_dict['Type'] = food_type
            sub_dict['Menu'] = food_menu
            sub_dic
            
             # total_dict에 담기
            total_dict[k] = sub_dict
            k += 1
            time.sleep(1)
            
            # 뒤로 가기
            driver.back()
            time.sleep(0.5)
            
    df = pd.DataFrame.from_dict(total_dict, orient='index')
    driver.close()
    return df